<a href="https://colab.research.google.com/github/cabroderick/ML-AM-MQP/blob/main/mask_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Testing Mask-RCNN on sample dataset provided from ME team**

In [ ]:
# uninstall improper package versions
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

In [ ]:
# reinstall with proper versions
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

In [ ]:
# import mask rcnn and set up
%cd
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!pip show mask-rcnn

In [ ]:
# import training data
%cd
!git clone https://github.com/cabroderick/ML-AM-MQP
!ls

In [ ]:
# list data contents
%cd '~/ML-AM-MQP/Data/Trial/H6'
!ls
%cd '~/ML-AM-MQP/Data/Trial/Labeled H6'
!ls
%cd

In [ ]:
# imports
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from cv2 import imread
import os
import json
import numpy as np
import urllib.request

In [ ]:
# configure network

class CustomConfig(Config):
    NAME = "object"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    # number of classes (including background)
    NUM_CLASSES = 1 + 2
    STEPS_PER_EPOCH = 100
    LEARNING_RATE = .001

config = CustomConfig()

# set up dataset

class AMDataset(utils.Dataset):
  def load_dataset(self, validation=False):
    IMAGES_DIR = "/root/ML-AM-MQP/Data/Trial/H6/"
    ANNOTATIONS_DIR = '/root/ML-AM-MQP/Data/Trial/Labeled H6/'
    IMG_WIDTH = 1280
    IMG_HEIGHT = 1024

    self.add_class('dataset', 1, 'gas porosity')
    self.add_class('dataset', 2, 'lack of fusion porosity')

    val_images = 5 # keeps track of images to reserve for validation set
    total_images = len(os.listdir(IMAGES_DIR))
    for filename in os.listdir(IMAGES_DIR):
      if validation and val_images > 0:
        val_images -=1
        continue
      if (not validation) and val_images < total_images:
        val_images += 1
        continue

      image_id = filename[:-4]
      image_path = IMAGES_DIR + image_id + '.tif'
      annotation_path = ANNOTATIONS_DIR + image_id + '_labeled.json'
      self.add_image('dataset',
                     image_id=image_id, 
                     path=image_path, 
                     annotation=annotation_path,
                     width=IMG_WIDTH,
                     height=IMG_HEIGHT)

  def load_mask(self, image_id):
    class_ids = list() # list of class ids corresponding to each mask in the mask list
    image_info = self.image_info[image_id] # extract image info from data added earlier

    width = image_info['width']
    height = image_info['height']
    path = image_info['annotation']

    masks_index = 0 # keep track of index for use in masks

    boxes = self.extract_boxes(path) # extract mask data from json file
    masks = np.zeros([height, width, len(boxes)], dtype='uint8') # initialize array of masks for each bounding box
    for i in range(len(boxes)):
      box = boxes[i]
      for key in box:
        if (box[key]): # make sure box is not empty
          col_s, col_e = int(box[key][0][0]), int(box[key][0][1])
          row_s, row_e = int(box[key][1][0]), int(box[key][1][1])
          masks[row_s:row_e, col_s:col_e, masks_index] = 1
          masks_index += 1
          class_ids.append(self.class_names.index(key))

    return masks, np.array(class_ids)

  def extract_boxes(self, filename): # helper to extract bounding boxes from json
      f = open(filename,)
      data = json.load(f)

      boxes = [] # store box coordinates in a dictionary corresponding to labels

      for i in data['shapes']:
        if i['shape_type'] == 'rectangle':
          box = {'gas porosity': [], 'lack of fusion porosity': []}
          label = i['label']
          box[label] = i['points']
          boxes.append(box)
 
      return boxes

# set up train and validation data

dataset_train = AMDataset()
dataset_train.load_dataset(validation=False)
dataset_train.prepare()

dataset_val = AMDataset()
dataset_val.load_dataset(validation=True)
dataset_val.prepare()

# configure model and load coco weights

urllib.request.urlretrieve("https://github.com/matterport/Mask_RCNN/releases/download/v1.0/mask_rcnn_coco.h5", "mask_rcnn_coco.h5")
model = MaskRCNN(mode='training', model_dir='./', config=CustomConfig())
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# train model
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=.001,
            epochs=1,
            layers='heads')

# save training results to external file
model_path = 'custom_maskrcnn_weights.h5'
model.keras_model.save_weights(model_path)

In [ ]:
# model_path = 'custom_maskrcnn_weights.h5'
# model.keras_model.save_weights(model_path)

# draw an image with detected objects
def draw_image_with_boxes(filename, boxes_list):
     # load the image
     data = pyplot.imread(filename)
     # plot the image
     pyplot.imshow(data)
     # get the context for drawing boxes
     ax = pyplot.gca()
     # plot each box
     for box in boxes_list:
          # get coordinates
          y1, x1, y2, x2 = box
          # calculate width and height of the box
          width, height = x2 - x1, y2 - y1
          # create the shape
          rect = Rectangle((x1, y1), width, height, fill=False, color='red')
          # draw the box
          ax.add_patch(rect)
     # show the plot
     pyplot.show()

# load photograph
img = load_img('/root/ML-AM-MQP/Data/Trial/H6/A1H6COL_10X_BF_ZUYL_27.tif')
img = img_to_array(img)

model = MaskRCNN(mode='inference', model_dir='./', config=CustomConfig())
model.load_weights('/root/ML-AM-MQP/Data/Trial/Labeled H6/custom_maskrcnn_weights.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# make prediction
results = model.detect(img, verbose=0)
# visualize the results
draw_image_with_boxes('/root/ML-AM-MQP/Data/Trial/H6/A1H6COL_10X_BF_ZUYL_27.tif', results[0]['rois'])

In [ ]:

# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
 
# draw an image with detected objects
def draw_image_with_boxes(filename, boxes_list):
     # load the image
     data = pyplot.imread(filename)
     # plot the image
     pyplot.imshow(data)
     # get the context for drawing boxes
     ax = pyplot.gca()
     # plot each box
     for box in boxes_list:
          # get coordinates
          y1, x1, y2, x2 = box
          # calculate width and height of the box
          width, height = x2 - x1, y2 - y1
          # create the shape
          rect = Rectangle((x1, y1), width, height, fill=False, color='red')
          # draw the box
          ax.add_patch(rect)
     # show the plot
     pyplot.show()
 
# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 3
 
# define the model
rcnn = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())
# load coco model weights
rcnn.load_weights('/root/ML-AM-MQP/Data/Trial/Labeled H6/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
rcnn.load_weights('/root/ML-AM-MQP/Data/Trial/Labeled H6/custom_maskrcnn_weights.h5', by_name=True)
# load photograph
img = load_img('/root/ML-AM-MQP/Data/Trial/H6/A1H6COL_10X_BF_ZUYL_22.tif')
img = img_to_array(img)
# make prediction
results = rcnn.detect([img], verbose=0)
# visualize the results
draw_image_with_boxes('/root/ML-AM-MQP/Data/Trial/H6/A1H6COL_10X_BF_ZUYL_22.tif', results[0]['rois'])